## Train model on black/white images

This is a quick experiment to see if removing the color channel entirely will improve performance with daytime/nighttime lighting.

## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Load Training CSV File

Displays dataframe for verification purposes.

In [2]:
# Load the training CSV file
csv_file = '../../image_labels/vertical_speed_labels/improved_vertical_speed_labels.csv'
training_df = pd.read_csv(csv_file)

image_directory = '../../media/vertical_speed_frames_expanded'
training_df['image_path'] = training_df['image_path'].apply(lambda x: os.path.join(image_directory, x))

training_df

,image_path,vertical_speed
0,../../media/vertical_speed_frames_expanded/day...,0.00
1,../../media/vertical_speed_frames_expanded/day...,0.00
2,../../media/vertical_speed_frames_expanded/day...,0.00
3,../../media/vertical_speed_frames_expanded/day...,0.00
4,../../media/vertical_speed_frames_expanded/day...,0.00
...,...,...
2098,../../media/vertical_speed_frames_expanded/day...,0.00
2099,../../media/vertical_speed_frames_expanded/day...,-0.25
2100,../../media/vertical_speed_frames_expanded/day...,-0.25
2101,../../media/vertical_speed_frames_expanded/day...,-0.25


## Helper Function - Load Image

In [3]:
def load_image(file_path):
    image = cv2.imread(file_path)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

## Load Images and apply labels from CSV

In [4]:
# Load images into a NumPy array
images = np.array([load_image(file_path) for file_path in training_df['image_path']])

target_column_name = 'vertical_speed'

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(training_df[target_column_name])

# Optionally, one-hot encode the labels (if using categorical cross-entropy)
num_classes = len(label_encoder.classes_)
one_hot_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes)


# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)

## Epoch Notes

Best for airspeed - Epoch 13

## Execute - Run Training Model

In [6]:
# Define the model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 125, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust num_classes as needed
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use appropriate loss function
              metrics=['accuracy'])

# Load and preprocess your dataset
# You can use TensorFlow's `ImageDataGenerator` for data augmentation and preprocessing.

# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Save the trained model
model.save('regenerated_vertical_speed_model_20_epoch_bw.h5')


Epoch 1/20
53/53 [==============================] - 4s 69ms/step - loss: 3.4519 - accuracy: 0.1534 - val_loss: 2.5976 - val_accuracy: 0.3373
Epoch 2/20
53/53 [==============================] - 4s 69ms/step - loss: 1.3761 - accuracy: 0.6356 - val_loss: 0.7254 - val_accuracy: 0.8029
Epoch 3/20
53/53 [==============================] - 4s 68ms/step - loss: 0.4566 - accuracy: 0.8460 - val_loss: 0.3667 - val_accuracy: 0.8860
Epoch 4/20
53/53 [==============================] - 4s 68ms/step - loss: 0.3195 - accuracy: 0.8805 - val_loss: 0.3319 - val_accuracy: 0.8860
Epoch 5/20
53/53 [==============================] - 4s 69ms/step - loss: 0.2709 - accuracy: 0.8989 - val_loss: 0.2246 - val_accuracy: 0.9145
Epoch 6/20
53/53 [==============================] - 4s 69ms/step - loss: 0.2485 - accuracy: 0.8971 - val_loss: 0.2890 - val_accuracy: 0.8955
Epoch 7/20
53/53 [==============================] - 4s 70ms/step - loss: 0.2449 - accuracy: 0.9049 - val_loss: 0.2199 - val_accuracy: 0.9264
Epoch 8/20
53